In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation 
from IPython.display import HTML
import time

In [7]:
print(lambda_min)
print(delta_lambda)
print(delta_ds)
print(delta_z)
print(dz)

9.881422924901186e-06
2.4703557312252967e-07
3.3499999999999997e-06
2.4703557312252967e-07
2.4662576687116563e-07


In [3]:
#=====User defined Values ============
ds = np.array([134,134,134]) #Layers' tickness vector in cm
eps = np.array([1, 1 , 1]) #Refraction index vector
mus = np.array([6.4009,2.25,1.9044]) #Refraction index vector
lamda_min = 250  #Minimun wave Lenght in nm
lamda_max = 1000 #Maximun wave Lenght in nm
Ny = 40 #Number of cells in a wavelength
Nd = 4 #Number of cells in the smallest layer
offset= 3+100 #adtional points to grid +3 for boundary TRN/REF/SRC +20 for space regions )
nbc = 1  # refraction index at boundary conditions
epsrc = 1 #  Permittivity at boundary source
musrc = 1 # Permeabilitya at boundary source
src_ini=2 # cell in wich the cell will be injected
frames = 500


#======Calculation of steps in the spatial yee grid========
ds = ds*(10**(-7))
c0= 299792458 *100 #speed of light cm/s
fmax = c0/(lamda_min*(10**(-7)))   #Max wave frecuency in Hz
fmin = c0/(lamda_max*(10**(-7)))   #Max wave frecuency in Hz
index = np.sqrt(eps*mus) #Refraction index vector
lambda_min = c0/(fmax*float(np.amax(index))) # minimun wavelength
delta_lambda=(lambda_min/Ny) #Distance of the cell taking into accont the wavelength in cm
delta_ds=np.amin(ds)/Nd #Distance of the cell taking into accont the layer
delta_z = np.amin([delta_lambda, delta_ds]) #Distance of the cell
d = np.sum(ds) #total distance of the space to model
N = int(np.ceil(d/delta_z)) #Total steps in grid 
dz = d/N
N += offset 
z = np.arange(0, N, 1)*dz #z axis

#========Compute Position of Materials on Grid========
eps_d=np.ones(N)
mus_d=np.ones(N)
init = int(np.ceil(offset/2))
for i in range(np.size(index)):
    temp=int(init+np.round(ds[i]/dz,0)+1)
    eps_d[init:temp]=eps[i]
    mus_d[init:temp]=mus[i]
    init=temp

#======Calculation of steps in the temporal yee grid========
dt=(nbc*dz)/(2*c0) #time of each step
tprop =  np.amax(index)*N*dz/c0 #total time it takes for a wave to propagate across the grid one time
tao = 1/(2*fmax)
T = 12*tao + 3*tprop #Total Number of Iterations
S = int(np.ceil(T/dt))
frames = int(np.ceil(S/frames))

#======Compute the Source Functions for Ey/Hx Mode ========
t = np.arange(0, S-1, 1)*dt #time axis
delt = nbc*dz/(2*c0) + dt/2 #total delay between E and H because Yee Grid
A = - np.sqrt(epsrc/musrc) #amplitude of H field
Esrc = np.exp(-((t-6*tao)/tao)**2) #E field source
Hsrc = A*np.exp(-((t-6*tao+delt)/tao)**2) #H field source


In [ ]:
plt.plot(t,Esrc)
plt.plot(t,Hsrc)

In [ ]:
tic = time.clock()

#======Initialize the Fourier Transforms ========
NFREQ = 1000;
FREQ = np.linspace(fmin,fmax,NFREQ);
K = np.exp(-1j*2*np.pi*dt*FREQ);
REF = np.zeros(NFREQ)+0j;
TRN = np.zeros(NFREQ)+0j;
SRC = np.zeros(NFREQ)+0j;
NREF = np.zeros(NFREQ)+0j;
NTRN = np.zeros(NFREQ)+0j;

#======Initialize update coficients and compensate for numerical dispertion ========
mE=c0*dt/eps_d
mH=c0*dt/mus_d

#======E and H ========
Ey=np.zeros(N)
Hx=np.zeros(N)
Eys=np.zeros((int(np.ceil((S-2)/frames)),N))
Hxs=np.zeros((int(np.ceil((S-2)/frames)),N))

#======Initialize PML values ========
h1 = 0
h2 = 0
e1 = 0
e2 = 0

for i in range(S-2):
    
    # Update PML values for H 
    h2 = h1
    h1 = Hx[0]
    
    # Applay PML values for H 
    Hx[-1] = Hx[-1] + mH[N-1]*(e2 - Ey[-1])/dz
    
    # Update H from E (Dirichlet Boundary Conditions)    
    Hx[0:N-1] = Hx[0:N-1] + mH[0:N-1]*(Ey[1:N] - Ey[0:N-1])/dz
    
    # Correction for Hx adjacent to TFSF boundary
    Hx[src_ini-1] -= mH[src_ini-1]*Esrc[i]/dz

    # Update PML values for E 
    e2 = e1
    e1 = Ey[-1]
    
    # Applay PML values for E 
    Ey[0] = Ey[0] + mE[0]*(Hx[0] - h2)/dz
    
    # Update E from H 
    Ey[1:N] = Ey[1:N] + (mE[1:N]*(Hx[1:N] - Hx[0:N-1]))/dz

    # Correction for Ey adjacent to TFSF boundary
    Ey[src_ini] -= mE[src_ini]*Hsrc[i]/dz

    #Computing discret fourier transform
    REF += (K**i)*Ey[1]
    TRN += (K**i)*Ey[-1]
    SRC += (K**i)*Esrc[i]
    
    if i % frames == 0:        
        Hxs[int(np.ceil(i/frames))]=Hx
        Eys[int(np.ceil(i/frames))]=Ey

NREF = np.absolute(REF/SRC)**2
NTRN = np.absolute(TRN/SRC)**2
CON = NREF+ NTRN

toc = time.clock()
print("Total time: ", toc - tic, " seconds")

In [ ]:
fig = plt.figure()
plts = []

for i in range(int(np.floor((S-2)/frames))):
    p, = plt.plot(z,Hxs[i], color="blue")
    p2, = plt.plot(z,Eys[i], '-r')
    plts.append( [p,p2] )     

#Add Layers Background
plt.axis([0, (N-1)*dz, -2, 2])
init = int(np.ceil(offset/2)) 
color=0.2
for j in range(np.size(index)):
    temp=int(init+np.round(ds[j]/dz,0)+1)
    plt.axvspan(init*dz, temp*dz, facecolor=str(color), alpha=0.5)
    init=temp
    color += 0.1
    
ani = animation.ArtistAnimation(fig, plts, interval=50, repeat_delay=3000)
HTML(ani.to_jshtml())

In [ ]:
#Ploting Fourier tranform for reflection and trasmition
lambd = (c0/FREQ)*(10**7)
lambd= np.flip(lambd, 0)

plt.subplot(211)
plt.plot(lambd, np.flip(np.absolute(TRN),0), label='TRN')
plt.legend()
plt.subplot(212)
plt.plot(lambd, np.flip(np.absolute(REF),0), label='REF')
plt.legend()

In [ ]:
#Ploting reflectance vs transmittanc vs transmittance

plt.plot(lambd, np.flip(CON,0), label='Conservation')
plt.plot(lambd, np.flip(NREF,0), label='Reflectance')
plt.plot(lambd, np.flip(NTRN,0), label='Transmittance')
plt.legend()

In [ ]:
(lambd==633.07984791).nonzero

In [ ]:
lambd[500]

In [ ]:
print(NREF[500])
print(NTRN[500])